## Без catboost никуда

In [1]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import zipfile
import os
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KDTree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import requests
import geopandas as gpd
warnings.filterwarnings('ignore')

In [2]:
def haversine(lat1, lon1, lat2=55.7558, lon2=37.6176):

    R = 6371.0

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

## все выше - загрузка датасета и импорт библиотек!!

In [3]:
train_data = pd.read_csv('data/train.tsv', sep='\t')
test_data = pd.read_csv('data/test.tsv', sep='\t')
reviews = pd.read_csv('data/reviews.txv/reviews.tsv', sep='\t')

In [4]:
y_train = train_data['target']

In [5]:
train_data.head(2)

,id,name,coordinates,category,address,target,traffic_300m,homes_300m,works_300m,female_300m,...,doramas_1000m,computer_components_1000m,humor_1000m,car_market_1000m,no_higher_education_1000m,goods_for_moms_and_babies_1000m,age_25-34_1000m,male_1000m,phone_repair_1000m,mean_income_1000m
0,1365,"Городская поликлиника № 109, филиал № 2","[37.735049, 55.719667]",health,"Грайвороновская ул., 18, корп. 1, Москва",4.1,75429,16113.582471,15756.246444,51316.0,...,4668.0,7718.0,33389.0,18306.0,426241.0,415.0,380148.0,619550.0,2781.0,113767.387249
1,8230,Wellness Club Nebo,"[37.537083, 55.749511]",swimming_pool,"Пресненская наб., 12, Москва",3.6,246535,8578.458740,31315.672794,192547.0,...,3431.0,11463.0,61107.0,23662.0,488685.0,356.0,436721.0,764733.0,4264.0,122931.921255


In [6]:
train_data[train_data['target'] < 1].shape[0]

3986

In [7]:
train_data['address'].value_counts()

address
Кировоградская ул., 13А, Москва                           59
Ходынский бул., 4, Москва                                 58
просп. Мира, 211, корп. 2, Москва                         52
площадь Киевского Вокзала, 2, Москва                      51
Дмитровское ш., 163А, Москва                              49
                                                          ..
Армавирская ул., 1/20, Москва                              1
Малый Толмачёвский пер., 8/11с3, Москва                    1
Железнодорожный пр., с27, микрорайон Керамик, Балашиха     1
Лётная ул., 27А, Мытищи                                    1
ул. Панфёрова, 8, корп. 2, Москва                          1
Name: count, Length: 22155, dtype: int64

# TF-IDF оказался полезнее BERT, прогнал еще через SVD

In [8]:

def extract_text_features(reviews_df):


    reviews_agg = reviews_df.groupby('id')['text'].agg(' '.join).reset_index()


    tfidf = TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 3),
        stop_words=['и', 'в', 'на', 'с', 'у', 'к', 'по', 'для', 'это', 'то', 'так']
    )
    tfidf_matrix = tfidf.fit_transform(reviews_agg['text'])


    svd = TruncatedSVD(n_components=100, random_state=42)
    tfidf_reduced = svd.fit_transform(tfidf_matrix)



    return reviews_agg, tfidf_reduced,
reviews_agg, tfidf_reduced =extract_text_features(reviews)
import pandas as pd

tfidf_array = tfidf_reduced
tfidf_df = pd.DataFrame(tfidf_array, index=reviews_agg.index)

tfidf_df.columns = [f'tfidf_{i}' for i in range(tfidf_df.shape[1])]

result = pd.concat([reviews_agg, tfidf_df], axis=1)


In [9]:
'''

def extract_bert_features(reviews_df, model_name="deeppavlov/rubert-base-cased"):


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    reviews_agg = reviews_df.groupby("id")["text"].agg(" ".join).reset_index()

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    embeddings = []

    for text in tqdm(reviews_agg["text"], desc="Extracting BERT embeddings"):
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=128,
            padding="max_length"
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

        embeddings.append(emb[0])

    bert_array = np.vstack(embeddings)
    bert_df = pd.DataFrame(bert_array, columns=[f"bert_{i}" for i in range(bert_array.shape[1])])

    result = pd.concat([reviews_agg, bert_df], axis=1)
    return reviews_agg, bert_df, result'''


'\n\ndef extract_bert_features(reviews_df, model_name="deeppavlov/rubert-base-cased"):\n\n\n    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\n    reviews_agg = reviews_df.groupby("id")["text"].agg(" ".join).reset_index()\n\n    tokenizer = AutoTokenizer.from_pretrained(model_name)\n    model = AutoModel.from_pretrained(model_name).to(device)\n    model.eval()\n\n    embeddings = []\n\n    for text in tqdm(reviews_agg["text"], desc="Extracting BERT embeddings"):\n        inputs = tokenizer(\n            text,\n            return_tensors="pt",\n            truncation=True,\n            max_length=128,\n            padding="max_length"\n        ).to(device)\n\n        with torch.no_grad():\n            outputs = model(**inputs)\n            emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()\n\n        embeddings.append(emb[0])\n\n    bert_array = np.vstack(embeddings)\n    bert_df = pd.DataFrame(bert_array, columns=[f"bert_{i}" for i in range(bert_array.s

In [10]:
'''reviews_agg_bert, bert_df, result_bert = extract_bert_features(reviews)

full_features = result.merge(result_bert.drop(columns=['text']), on='id', how='left')'''

"reviews_agg_bert, bert_df, result_bert = extract_bert_features(reviews)\n\nfull_features = result.merge(result_bert.drop(columns=['text']), on='id', how='left')"

In [11]:
reviews_agg, tfidf_reduced = extract_text_features(reviews)

In [12]:

tfidf_array = tfidf_reduced
tfidf_df = pd.DataFrame(tfidf_array, index=reviews_agg.index)

tfidf_df.columns = [f'tfidf_{i}' for i in range(tfidf_df.shape[1])]

result = pd.concat([reviews_agg, tfidf_df], axis=1)


In [13]:
'''
assert 'id' in result.columns and 'id' in result_bert.columns

text_features_full = (
    result
    .drop(columns=['text'], errors='ignore')
    .merge(
        result_bert.drop(columns=['text'], errors='ignore'),
        on='id',
        how='outer'
    )
)
'''

"\nassert 'id' in result.columns and 'id' in result_bert.columns\n\ntext_features_full = (\n    result\n    .drop(columns=['text'], errors='ignore')\n    .merge(\n        result_bert.drop(columns=['text'], errors='ignore'),\n        on='id',\n        how='outer'\n    )\n)\n"

# Работает с основной частью данных, развертываем coordinates

In [14]:
train_data = train_data[train_data['target'] >= 1]

In [15]:
train_data[['longitude', 'latitude']] = train_data['coordinates'].str.strip('[]').str.split(',', expand=True).astype(float)
test_data[['longitude', 'latitude']] = test_data['coordinates'].str.strip('[]').str.split(',', expand=True).astype(float)

train_data['distance_to_moscow_center_km'] = train_data.apply(
    lambda row: haversine(row['latitude'], row['longitude']), axis=1
)
test_data['distance_to_moscow_center_km'] = test_data.apply(
    lambda row: haversine(row['latitude'], row['longitude']), axis=1
)



# Берем абсолютно все из текста, мне это в итоге помогло

In [16]:

def add_review_features(main_data, reviews_file='/content/reviews.txv/reviews.tsv'):

    try:
        reviews = pd.read_csv(reviews_file, sep='\t')

        reviews['text_length'] = reviews['text'].str.len().fillna(0)
        reviews['word_count'] = reviews['text'].str.split().str.len().fillna(0)
        reviews['char_count'] = reviews['text'].str.replace(' ', '').str.len().fillna(0)

        reviews['exclamation_count'] = reviews['text'].str.count('!').fillna(0)
        reviews['question_count'] = reviews['text'].str.count('\?').fillna(0)
        reviews['capital_ratio'] = reviews['text'].apply(
            lambda x: sum(1 for char in str(x) if char.isupper()) / len(str(x)) if len(str(x)) > 0 else 0
        )

        reviews['comma_count'] = reviews['text'].str.count(',').fillna(0)
        reviews['dot_count'] = reviews['text'].str.count('\.').fillna(0)
        reviews['ellipsis_count'] = reviews['text'].str.count('\.\.\.').fillna(0)
        reviews['line_breaks_count'] = reviews['text'].str.count('\n').fillna(0)
        reviews['has_quotes'] = reviews['text'].str.contains('"|\'').fillna(0).astype(int)

        def simple_sentiment_analysis(text):
            if pd.isna(text):
                return 0
            text = str(text).lower()
            positive_words = ['хорош', 'отличн', 'прекрасн', 'рекоменд', 'супер',
                            'класс', 'любим', 'удовольств', 'замечательн', 'восхитительн',
                            'быстро', 'вежлив', 'чист', 'комфортн', 'вкусн', 'спасибо',
                            'благодар', 'совету', 'порадова', 'понрави']
            negative_words = ['плох', 'ужасн', 'кошмар', 'разочарован', 'не рекоменд',
                            'отвратительн', 'груб', 'грязн', 'дорог', 'долго',
                            'медленно', 'невкусн', 'шумн', 'тесн', 'обман', 'жаль',
                            'напугал', 'отврат', 'противн', 'увол']

            pos_count = sum(1 for word in positive_words if word in text)
            neg_count = sum(1 for word in negative_words if word in text)

            total = pos_count + neg_count
            if total > 0:
                return (pos_count - neg_count) / total
            return 0

        reviews['sentiment'] = reviews['text'].apply(simple_sentiment_analysis)


        def get_sentiment_categories(text):
            if pd.isna(text):
                return 0, 0, 0
            text = str(text).lower()


            service_words = ['обслуживан', 'персонал', 'сотрудник', 'администратор', 'официант']
            quality_words = ['качеств', 'уровен', 'стандарт', 'профессионал']
            price_words = ['цен', 'стоим', 'дорог', 'дешев', 'выгодн']
            atmosphere_words = ['атмосфер', 'интерьер', 'уютн', 'комфорт', 'обстановк']

            service_count = sum(1 for word in service_words if word in text)
            quality_count = sum(1 for word in quality_words if word in text)
            price_count = sum(1 for word in price_words if word in text)
            atmosphere_count = sum(1 for word in atmosphere_words if word in text)

            total_mentions = service_count + quality_count + price_count + atmosphere_count
            if total_mentions > 0:
                return service_count/total_mentions, quality_count/total_mentions, price_count/total_mentions
            return 0, 0, 0

        reviews[['service_mention_ratio', 'quality_mention_ratio', 'price_mention_ratio']] = \
            reviews['text'].apply(lambda x: pd.Series(get_sentiment_categories(x)))

        reviews['unique_words_ratio'] = reviews['text'].apply(
            lambda x: len(set(str(x).split())) / len(str(x).split()) if len(str(x).split()) > 0 else 0
        )

        reviews['avg_word_length'] = reviews['char_count'] / reviews['word_count'].replace(0, 1)
        reviews['has_emoji'] = reviews['text'].str.contains('[^\x00-\x7F]').fillna(0).astype(int)

        reviews['long_words_ratio'] = reviews['text'].apply(
            lambda x: sum(1 for word in str(x).split() if len(word) > 6) / len(str(x).split())
            if len(str(x).split()) > 0 else 0
        )

        reviews['sentence_count'] = reviews['text'].str.count('[.!?]+').fillna(0)
        reviews['avg_sentence_length'] = reviews['word_count'] / reviews['sentence_count'].replace(0, 1)

        review_stats = reviews.groupby('id').agg({
            'text': 'count',
            'text_length': ['mean', 'std', 'max', 'min'],
            'word_count': ['mean', 'std', 'max', 'min'],
            'char_count': ['mean', 'std'],
            'exclamation_count': ['mean', 'sum', 'max'],
            'question_count': ['mean', 'sum', 'max'],
            'capital_ratio': 'mean',
            'comma_count': 'mean',
            'dot_count': 'mean',
            'ellipsis_count': 'sum',
            'line_breaks_count': 'mean',
            'has_quotes': 'mean',
            'sentiment': ['mean', 'std', 'min', 'max', lambda x: (x > 0.1).sum(), lambda x: (x < -0.1).sum()],
            'service_mention_ratio': 'mean',
            'quality_mention_ratio': 'mean',
            'price_mention_ratio': 'mean',
            'unique_words_ratio': 'mean',
            'avg_word_length': 'mean',
            'has_emoji': 'mean',
            'long_words_ratio': 'mean',
            'sentence_count': 'mean',
            'avg_sentence_length': 'mean'
        }).reset_index()

        review_stats.columns = [
            'id',
            'review_count',
            'avg_text_length', 'std_text_length', 'max_text_length', 'min_text_length',
            'avg_word_count', 'std_word_count', 'max_word_count', 'min_word_count',
            'avg_char_count', 'std_char_count',
            'avg_exclamation', 'total_exclamation', 'max_exclamation',
            'avg_question', 'total_question', 'max_question',
            'avg_capital_ratio',
            'avg_comma_count',
            'avg_dot_count',
            'total_ellipsis',
            'avg_line_breaks',
            'quotes_ratio',
            'avg_sentiment', 'std_sentiment', 'min_sentiment', 'max_sentiment', 'positive_reviews_count', 'negative_reviews_count',
            'avg_service_mention',
            'avg_quality_mention',
            'avg_price_mention',
            'avg_unique_words_ratio',
            'avg_word_length',
            'emoji_ratio',
            'avg_long_words_ratio',
            'avg_sentence_count',
            'avg_sentence_length'
        ]
        review_stats['sentiment_volatility'] = review_stats['std_sentiment'].fillna(0)
        review_stats['text_length_variation'] = review_stats['std_text_length'] / review_stats['avg_text_length'].replace(0, 1)
        review_stats['text_length_variation'] = review_stats['text_length_variation'].replace([np.inf, -np.inf], 0)
        review_stats['emotional_intensity'] = (review_stats['avg_exclamation'] + review_stats['avg_question']) / 2
        review_stats['review_engagement'] = (review_stats['avg_text_length'] * review_stats['emotional_intensity'])
        review_stats['has_positive_reviews'] = (review_stats['max_sentiment'] > 0.1).astype(int)
        review_stats['has_negative_reviews'] = (review_stats['min_sentiment'] < -0.1).astype(int)
        review_stats['has_mixed_reviews'] = ((review_stats['max_sentiment'] > 0.1) &
                                           (review_stats['min_sentiment'] < -0.1)).astype(int)

        review_stats['positive_negative_ratio'] = review_stats['positive_reviews_count'] / (review_stats['negative_reviews_count'] + 1)
        review_stats['sentiment_balance'] = (review_stats['positive_reviews_count'] - review_stats['negative_reviews_count']) / review_stats['review_count'].replace(0, 1)
        review_stats['text_complexity_index'] = (review_stats['avg_long_words_ratio'] + review_stats['avg_unique_words_ratio']) / 2
        review_stats['punctuation_diversity'] = (review_stats['avg_exclamation'] + review_stats['avg_question'] + review_stats['avg_comma_count']) / 3
        review_stats['has_detailed_reviews'] = (review_stats['avg_word_count'] > 20).astype(int)
        review_stats['has_emotional_reviews'] = (review_stats['emotional_intensity'] > 0.5).astype(int)
        review_stats['has_structured_reviews'] = (review_stats['avg_sentence_count'] > 2).astype(int)

        main_data = main_data.merge(review_stats, on='id', how='left')

        review_columns = [col for col in review_stats.columns if col != 'id']
        for col in review_columns:
            if 'count' in col or 'total' in col or 'max' in col or 'min' in col:
                main_data[col] = main_data[col].fillna(0)
            elif col.startswith('has_') or col.endswith('_ratio') or 'balance' in col or 'index' in col:
                main_data[col] = main_data[col].fillna(0)
            else:
                main_data[col] = main_data[col].fillna(0)



        return main_data

    except Exception as e:


        base_features = [
            'review_count', 'avg_text_length', 'std_text_length', 'max_text_length', 'min_text_length',
            'avg_word_count', 'std_word_count', 'max_word_count', 'min_word_count', 'avg_char_count',
            'std_char_count', 'avg_exclamation', 'total_exclamation', 'max_exclamation', 'avg_question',
            'total_question', 'max_question', 'avg_capital_ratio', 'avg_comma_count', 'avg_dot_count',
            'total_ellipsis', 'avg_line_breaks', 'quotes_ratio', 'avg_sentiment', 'std_sentiment',
            'min_sentiment', 'max_sentiment', 'positive_reviews_count', 'negative_reviews_count',
            'avg_service_mention', 'avg_quality_mention', 'avg_price_mention', 'avg_unique_words_ratio',
            'avg_word_length', 'emoji_ratio', 'avg_long_words_ratio', 'avg_sentence_count',
            'avg_sentence_length', 'sentiment_volatility', 'text_length_variation', 'emotional_intensity',
            'review_engagement', 'has_positive_reviews', 'has_negative_reviews', 'has_mixed_reviews',
            'positive_negative_ratio', 'sentiment_balance', 'text_complexity_index', 'punctuation_diversity',
            'has_detailed_reviews', 'has_emotional_reviews', 'has_structured_reviews'
        ]

        for feature in base_features:
            main_data[feature] = 0

        return main_data


In [17]:
train_data = add_review_features(train_data)
test_data = add_review_features(test_data)


# Хочется ввести балльную систему по доходам

In [18]:
def fqs(df):
  income_weights = {
      'below_average_income_1000m': 1,
      'average_income_1000m': 2,
      'above_average_income_1000m': 3,
      'high_income_1000m': 4,
      'premium_income_1000m': 5
  }

  df['wealth_numerator_1000m'] = sum(
      df[col] * weight for col, weight in income_weights.items()
  )

  df['wealth_denominator_1000m'] = df[
      list(income_weights.keys())
  ].sum(axis=1) + 1e-6


  df['wealth_index_1000m'] = df['wealth_numerator_1000m'] / df['wealth_denominator_1000m']

  df.drop(['wealth_numerator_1000m', 'wealth_denominator_1000m'], axis=1, inplace=True)
  return df


In [19]:
train_data = fqs(train_data)
test_data = fqs(test_data)

In [20]:
category_freq = pd.concat([train_data['category'], test_data['category']], axis=0).value_counts()
train_data['category_freq'] = train_data['category'].map(category_freq)
test_data['category_freq'] = test_data['category'].map(category_freq).fillna(0)

In [21]:
category_target_mean = train_data.groupby('category')['target'].mean()
train_data['category_target_enc'] = train_data['category'].map(category_target_mean)
test_data['category_target_enc'] = test_data['category'].map(category_target_mean).fillna(train_data['target'].mean())


In [22]:
coords_all = pd.concat([
    train_data[["longitude", "latitude"]],
    test_data[["longitude", "latitude"]]
], axis=0, ignore_index=True)


N_CLUSTERS = 100
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init="auto")


coords_all["cluster"] = kmeans.fit_predict(coords_all)

train_data["cluster"] = coords_all["cluster"].iloc[:len(train_data)].values
test_data["cluster"] = coords_all["cluster"].iloc[len(train_data):].values


cluster_stats = (
    train_data.groupby("cluster")["target"]
    .agg([
        ("mean_target_in_cluster", "mean"),
        ("std_target_in_cluster", "std"),
        ("count_in_cluster", "count"),
        ("median_target_in_cluster", "median"),
    ])
    .reset_index()
)

train_data = train_data.merge(cluster_stats, on="cluster", how="left")
test_data = test_data.merge(cluster_stats, on="cluster", how="left")

train_data["std_target_in_cluster"].fillna(0, inplace=True)
test_data["std_target_in_cluster"].fillna(0, inplace=True)



In [23]:
from sklearn.neighbors import BallTree

In [24]:

def add_density_features(df):

    coords = np.radians(df[['latitude', 'longitude']].values)

    tree = BallTree(coords, metric='haversine')

    r_300m = 0.3 / 6371
    r_1000m = 1.0 / 6371

    count_300m = tree.query_radius(coords, r=r_300m, count_only=True)
    count_1000m = tree.query_radius(coords, r=r_1000m, count_only=True)

    df['density_300m'] = count_300m - 1
    df['density_1000m'] = count_1000m - 1

    return df

In [25]:
def add_same_category_features(df):

    R = 6371
    r_300m = 0.3 / R
    r_1000m = 1.0 / R

    df['latitude_rad'] = np.radians(df['latitude'])
    df['longitude_rad'] = np.radians(df['longitude'])


    df['nearest_same_category_dist'] = np.nan
    df['same_category_count_300m'] = 0
    df['same_category_count_1000m'] = 0

    for cat in df['category'].unique():
        mask = df['category'] == cat
        coords = np.vstack((df.loc[mask, 'latitude_rad'], df.loc[mask, 'longitude_rad'])).T

        if len(coords) < 2:
            df.loc[mask, ['nearest_same_category_dist',
                          'same_category_count_300m',
                          'same_category_count_1000m']] = 0
            continue


        tree = BallTree(coords, metric='haversine')


        dist, _ = tree.query(coords, k=2)
        nearest_dist = dist[:, 1] * R * 1000


        count_300m = tree.query_radius(coords, r=r_300m, count_only=True) - 1
        count_1000m = tree.query_radius(coords, r=r_1000m, count_only=True) - 1

        df.loc[mask, 'nearest_same_category_dist'] = nearest_dist
        df.loc[mask, 'same_category_count_300m'] = count_300m
        df.loc[mask, 'same_category_count_1000m'] = count_1000m

    return df


In [26]:
concatef = pd.concat([train_data.drop(columns=['target']), test_data], axis=0)

In [27]:
concatef = add_density_features(concatef)
concatef = add_same_category_features(concatef)


address_counts = concatef['address'].value_counts()
name_counts = concatef['name'].value_counts()
category_counts = concatef['category'].value_counts()

concatef['address_count'] = concatef['address'].map(address_counts)
concatef['name_count'] = concatef['name'].map(name_counts)
concatef['category_count'] = concatef['category'].map(category_counts)



In [28]:
train_data = pd.concat([concatef[:train_data.shape[0]], train_data['target']], axis=1)
test_data = concatef[train_data.shape[0]:]

In [29]:
train_data.isna().sum()[train_data.isna().sum() > 0]

address          20
address_count    20
dtype: int64

In [30]:
url = "https://api.hh.ru/metro/1"
response = requests.get(url)
if response.status_code == 200:
    metro_data = response.json()
metros = []
for line in range(len(metro_data['lines'])):
    here = metro_data['lines'][line]['stations']
    for station in range(len(here)):
        metros.append([here[station]['name'], here[station]['lng'], here[station]['lat']])

metros = pd.DataFrame(metros, columns=['metro_name', 'lng', 'lat'])
geo_metros = gpd.GeoDataFrame(
    metros,
    geometry=gpd.points_from_xy(metros.lng, metros.lat),
    crs="EPSG:4326"
)
geo_metros = geo_metros.drop_duplicates(subset=['metro_name']).reset_index(drop=True)


In [31]:
def add_nearest_metros(df, geo_metros, n_neighbors=3):
    df = df.copy()
    df[['lng', 'lat']] = df['coordinates'].apply(lambda x: pd.Series(eval(x)))
    geo_df = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.lng, df.lat),
        crs="EPSG:4326"
    )
    metros_radians = np.deg2rad(geo_metros[['lat', 'lng']])
    places_radians = np.deg2rad(geo_df[['lat', 'lng']])

    tree = BallTree(metros_radians, metric='haversine')
    dist, ind = tree.query(places_radians, k=n_neighbors)
    dist_km = dist * 6371
    for i in range(n_neighbors):
        geo_df[f'metro_{i+1}_name'] = geo_metros.iloc[ind[:, i]]['metro_name'].values
        geo_df[f'metro_{i+1}_dist_km'] = dist_km[:, i]
    geo_df = geo_df.drop(columns='geometry')
    return geo_df

# auto ml

In [32]:
from autofeat import AutoFeatRegressor

# 1. Берем числовые колонки
num_cols = train_data.select_dtypes(include=['int', 'float']).columns.tolist()
# исключаем target и, если хочешь, геокоординаты в радианах
num_cols = [c for c in num_cols if c not in ['target', 'latitude_rad', 'longitude_rad', 'id', 'address_count', 'name_count', 'category_count']]

In [33]:
# train_data2 = train_data.copy().sample(2000)
X = train_data[num_cols]
X = X[X.columns[~X.columns.str.endswith('_1000m')][:100]]
y = train_data['target']

# 2. Создаем и обучаем AutoFeat
model = AutoFeatRegressor(
    feateng_steps=2,
    transformations=["log", "sqrt", "1/", "abs"], 
    verbose=2
)

# Генерируем новые признаки
X_new = model.fit_transform(X, y)

# Добавляем новые признаки в исходный датафрейм
# (исходные колонки не трогаем, добавляем только новые)
new_features = [col for col in X_new.columns if col not in X.columns]
for col in new_features:
    train_data[col] = X_new[col]

# Проверим результат
print(f"Добавлено {len(new_features)} новых фичей")
print("Теперь в train_data:", train_data.shape[1], "столбцов")

2025-11-12 22:31:11,833 INFO: [AutoFeat] The 2 step feature engineering process could generate up to 80200 features.
2025-11-12 22:31:11,834 INFO: [AutoFeat] With 37119 data points this new feature matrix would use about 11.91 gb of space.
2025-11-12 22:31:11,857 INFO: [feateng] Step 1: transformation of original features


2025-11-12 22:31:13,421 INFO: [feateng] Generated 100 transformed features from 100 original features - done.
2025-11-12 22:31:13,439 INFO: [feateng] Step 2: first combination of features


2025-11-12 22:31:41,391 INFO: [feateng] Generated 19900 feature combinations from 19900 original feature tuples - done.
2025-11-12 22:31:48,403 INFO: [feateng] Generated altogether 20100 new features in 2 steps
2025-11-12 22:31:48,403 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2025-11-12 22:32:02,127 INFO: [feateng] Generated a total of 16797 additional features


[featsel] Scaling data...

2025-11-12 22:32:05,442 INFO: [featsel] Feature selection run 1/5


...

2025-11-12 22:37:09,111 INFO: 
[featsel]	 Selected  87 features after noise filtering.
2025-11-12 22:37:09,180 INFO: [featsel] Feature selection run 5/5
2025-11-12 22:37:30,277 INFO: [featsel]	 67 initial features.


2025-11-12 22:38:21,854 INFO: 
[featsel]	 Selected  68 features after noise filtering.
2025-11-12 22:38:21,920 INFO: [featsel] 170 features after 5 feature selection runs
2025-11-12 22:38:33,623 INFO: [featsel] 13 features after correlation filtering
2025-11-12 22:38:33,858 INFO: [featsel] 10 features after noise filtering
2025-11-12 22:38:33,883 INFO: [AutoFeat] Computing 5 new features.


2025-11-12 22:38:34,724 INFO: [AutoFeat]     5/    5 new features ...done.


2025-11-12 22:38:34,741 INFO: [AutoFeat] Final dataframe with 105 feature columns (5 new).
2025-11-12 22:38:34,742 INFO: [AutoFeat] Training final regression model.
2025-11-12 22:38:34,860 INFO: [AutoFeat] Trained model: largest coefficients:
2025-11-12 22:38:34,861 INFO: 3.7730978073268986
2025-11-12 22:38:34,863 INFO: -0.000661 * childrens_transport_300m
2025-11-12 22:38:34,863 INFO: -0.000149 * jewelry_pawnshops_300m
2025-11-12 22:38:34,866 INFO: [AutoFeat] Final score: 0.0080


Добавлено 5 новых фичей
Теперь в train_data: 364 столбцов


In [36]:
X_test_new = model.transform(test_data[X.columns])

for col in new_features:
    test_data[col] = X_test_new[col]

2025-11-12 22:39:18,425 INFO: [AutoFeat] Computing 5 new features.
2025-11-12 22:39:18,437 INFO: [AutoFeat]     5/    5 new features ...done.


# merge

In [40]:
geo_cols = ["traffic_300m", "homes_300m", "works_300m", "mean_income_300m"]
dem_cols = ["female_300m", "male_300m", "age_25-34_300m", "employed_300m", "higher_education_300m"]

cluster_geo_dem = train_data.groupby("cluster")[geo_cols + dem_cols].mean().add_prefix("cluster_mean_")
train_data = train_data.merge(cluster_geo_dem, on="cluster", how="left")
test_data = test_data.merge(cluster_geo_dem, on="cluster", how="left")

In [44]:
tfidf_columns = [col for col in result.columns if col != 'id']

train_with_tfidf = train_data.merge(result, on='id', how='left')
test_with_tfidf = test_data.merge(result, on='id', how='left')

train_with_tfidf[tfidf_columns] = train_with_tfidf[tfidf_columns].fillna(0)
test_with_tfidf[tfidf_columns] = test_with_tfidf[tfidf_columns].fillna(0)

In [45]:
train_with_tfidf = add_nearest_metros(train_with_tfidf, geo_metros)
test_with_tfidf = add_nearest_metros(test_with_tfidf, geo_metros)

In [ ]:
sum(test_with_tfidf.isna().sum())

10

In [ ]:
train_with_tfidf['train_ticket_order_300m']

,train_ticket_order_300m
0,734.0
1,4701.0
2,1204.0
3,540.0
4,2411.0
...,...
37114,638.0
37115,1478.0
37116,1209.0
37117,981.0


In [ ]:
'''np.array(train_with_tfidf.columns)'''

'np.array(train_with_tfidf.columns)'

In [ ]:
# !pip install -U autogluon.tabular lightgbm




# val split

In [46]:
train_with_tfidf = train_with_tfidf.loc[:, ~train_with_tfidf.columns.duplicated()]
test_with_tfidf = test_with_tfidf.loc[:, ~test_with_tfidf.columns.duplicated()]


numeric_features = []
for col in train_with_tfidf.columns:
    if col not in ['id', 'category', 'coordinates', 'target']:
        if train_with_tfidf[col].dtype in ['int64', 'float64']:
            numeric_features.append(col)

categorical_features = [
    'category', 'metro_1_name', 'metro_2_name', 'metro_3_name',
    'name', 'address'
]

X = train_with_tfidf[numeric_features + categorical_features]
y = train_with_tfidf['target']
X_test = test_with_tfidf[numeric_features + categorical_features]

print(X.isna().sum()[X.isna().sum() > 0])

X[numeric_features] = X[numeric_features].fillna(X[numeric_features].median())
X_test[numeric_features] = X_test[numeric_features].fillna(X[numeric_features].median())


X[categorical_features] = X[categorical_features].fillna('unknown')
X_test[categorical_features] = X_test[categorical_features].fillna('unknown')

print(X.isna().sum()[X.isna().sum() > 0])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.20, random_state=42
)

address_count    20
address          20
dtype: int64
Series([], dtype: int64)


# Переходим к моделям, обучаем CatBoost, LightGBM и блендим их

In [47]:
from catboost import CatBoostRegressor

In [48]:
model = CatBoostRegressor(
    # iterations=15000,
    # learning_rate=0.01,
    loss_function='MAE',
    random_state=42,
    verbose=100,
    task_type='GPU',
    cat_features=categorical_features, # ВСЕ ЕЩЕ НЕ ОПТИМАЛЬНЫЕ ГИПЕРПАРАМЕТРЫ!!, МОЖНО ПОЛУЧШЕ
)
model.fit(
     X_train, y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=100,
    verbose=100, cat_features=categorical_features,
)

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.3272466	test: 0.3236733	best: 0.3236733 (0)	total: 83.3ms	remaining: 1m 23s
100:	learn: 0.2551648	test: 0.2494563	best: 0.2494563 (100)	total: 6.81s	remaining: 1m
200:	learn: 0.2452769	test: 0.2426939	best: 0.2426939 (200)	total: 13.4s	remaining: 53.3s
300:	learn: 0.2391134	test: 0.2396415	best: 0.2396415 (300)	total: 19.8s	remaining: 46s
400:	learn: 0.2342317	test: 0.2378991	best: 0.2378991 (400)	total: 26.5s	remaining: 39.6s
500:	learn: 0.2297291	test: 0.2367584	best: 0.2367584 (500)	total: 33.1s	remaining: 32.9s
600:	learn: 0.2262190	test: 0.2361290	best: 0.2361237 (596)	total: 39.6s	remaining: 26.3s
700:	learn: 0.2228503	test: 0.2357366	best: 0.2357342 (699)	total: 46.4s	remaining: 19.8s
800:	learn: 0.2199570	test: 0.2354506	best: 0.2354403 (786)	total: 52.9s	remaining: 13.1s
900:	learn: 0.2171743	test: 0.2351277	best: 0.2351193 (899)	total: 59.7s	remaining: 6.57s
999:	learn: 0.2142895	test: 0.2348912	best: 0.2348906 (997)	total: 1m 6s	remaining: 0us
bestTest = 0.234890

In [ ]:
# model.fit(X, y, cat_features=categorical_features, verbose=1000)

In [ ]:
# from lightgbm import LGBMRegressor

In [ ]:
# for col in categorical_features:
#     X[col] = X[col].astype('category')
# for col in categorical_features:
#     X_test[col] = X_test[col].astype('category')

In [ ]:
# from lightgbm import LGBMRegressor

# model_lgb = LGBMRegressor(
#     n_estimators=4000,
#     learning_rate=0.3, #АНАЛОГИЧНО И ЗДЕСЬ
#     objective='mae',
#     random_state=42,
#     verbose=-1,
# )

# model_lgb.fit(
#     X, y,
#     categorical_feature=categorical_features
# )

In [49]:
feature_names = X_train.columns
importances = model.get_feature_importance()

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importance_df.head(30))

                    feature  importance
337     category_target_enc    9.672144
366                 tfidf_0    8.958560
475                    name    8.098007
471                category    5.093202
350              name_count    4.523743
373                 tfidf_7    3.573103
374                 tfidf_8    3.143844
372                 tfidf_6    2.637194
369                 tfidf_3    2.577081
338  mean_target_in_cluster    1.789365
367                 tfidf_1    1.619506
368                 tfidf_2    1.540284
387                tfidf_21    1.447967
379                tfidf_13    1.420164
391                tfidf_25    1.376742
403                tfidf_37    1.355710
388                tfidf_22    1.287633
375                 tfidf_9    1.150914
370                 tfidf_4    1.140254
400                tfidf_34    0.893852
406                tfidf_40    0.835985
411                tfidf_45    0.827778
377                tfidf_11    0.809997
376                tfidf_10    0.785897


In [50]:
X_test = X_test.fillna('unknown')

In [51]:
pred = model.predict(X_test)

In [ ]:
pred_q = model_lgb.predict(X_test)

In [ ]:
overall_pred = pred * 0.9 + pred_q * 0.1 # тут тоже можно настраивать веса!!!

In [52]:
submission = pd.DataFrame({
    'id': test_data['id'],
    'target': pred
}).to_csv('Our_final_answer_with_auto_ml.csv', index=False)